[🥭 Mango Markets](https://mango.markets/) support is available at: [Docs](https://docs.mango.markets/) | [Discord](https://discord.gg/67jySBhxrg) | [Twitter](https://twitter.com/mangomarkets) | [Github](https://github.com/blockworks-foundation) | [Email](mailto:hello@blockworks.foundation)

[![Binder](https://mybinder.org/badge_logo.svg)](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowPrice.ipynb) [Run this code](https://mybinder.org/v2/gh/blockworks-foundation/mango-explorer-examples/HEAD?labpath=ShowPrice.ipynb) on Binder.

_🏃‍♀️ To run this notebook press the ⏩ icon in the toolbar above._

# 🥭 Show Streaming Perp Events

This notebook shows how to stream the event queue for a perp market via a websocket, showing new events as they appear.

In [ ]:
import mango
import rx
import threading

# Create a 'devnet' Context
context = mango.ContextBuilder.build(cluster_name="devnet")

# Load the market
stub = context.market_lookup.find_by_symbol("BTC-PERP")
market = mango.ensure_market_loaded(context, stub)

# Pick out the event queue address
event_queue_address = market.underlying_perp_market.event_queue
initial: mango.PerpEventQueue = mango.PerpEventQueue.load(context, event_queue_address, market.lot_size_converter)

perp_splitter: mango.UnseenPerpEventChangesTracker = mango.UnseenPerpEventChangesTracker(initial)
event_splitting_subscription = mango.WebSocketAccountSubscription(context, event_queue_address, lambda account_info: mango.PerpEventQueue.parse(account_info, market.lot_size_converter))

manager = mango.IndividualWebSocketSubscriptionManager(context)
manager.add(event_splitting_subscription)

publisher = event_splitting_subscription.publisher.pipe(rx.operators.flat_map(perp_splitter.unseen))
event_subscription = publisher.subscribe(mango.PrintingObserverSubscriber(False))

print("About to start streaming events. Press the stop button (⏹️) in the toolbar to stop streaming.")

manager.open()

# Wait - don't exit. Exiting will be handled by signals/interrupts.
waiter = threading.Event()
try:
    waiter.wait()
except:
    pass

event_subscription.dispose()
manager.dispose()

print("Example complete.")